# 한국어의 특징

## 한국어는 문법상 동사의 활용이 난해하고 조사의 존재 때문에 고립어인 영어보다 형태소 단위로 AI에게 학습시키기 난해하다.
## 하지만 영어보다는 문자를 발음할 때의 규칙성이 더 높기 때문에 STT/TTS를 제작할 때는 음절 단위로 학습시키는게 더 유리하다.


# 이를 위한 함수 chr(), ord()
## chr() : 문자 하나를 유니코드 숫자 값으로 변환시키는 함수
#### 예1) chr('A') == 65
#### 예2) chr('가') == 0xAC00
## ord() : 숫자를 해당 유니코드 숫자 값을 가진 문자 하나로 변환시키는 함수
#### 예1) ord(97) == 'a'


# 한국어 유니코드 완성형의 특징
## 한국어 유니코드는 완성형으로 숫자 값이 배정되어 있으나 숫자값 배정에 규칙이 존재해서 초성, 중성, 종성 분리가 쉽다
## 공식 : (초성 * 21 + 중성) * 28 + 종성 + 0xAC00
#### 초성 :
['ㄱ','ㄲ','ㄴ','ㄷ','ㄸ','ㄹ','ㅁ','ㅂ','ㅃ','ㅅ','ㅆ','ㅇ','ㅈ','ㅉ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ'] (19개)
#### 중성 :
['ㅏ','ㅐ','ㅑ','ㅒ','ㅓ','ㅔ','ㅕ','ㅖ','ㅗ','ㅘ','ㅙ','ㅚ','ㅛ','ㅜ','ㅝ','ㅞ','ㅟ','ㅠ','ㅡ','ㅢ','ㅣ'] (21개)
#### 종성 : 
['','ㄱ','ㄲ','ㄳ','ㄴ','ㄵ','ㄶ','ㄷ','ㄹ','ㄺ','ㄻ','ㄼ','ㄽ','ㄾ','ㄿ','ㅀ','ㅁ','ㅂ','ㅄ','ㅅ','ㅆ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ'] (28개)
#### 기타 특문을 위한 라벨링 :
['pad','unk',' ','.','!','?'] (7개, 가변적)


## 이를 위한 Multi-hot-encoding
### 기존 One-hot-encoding과 달리 1의 값이 여러개인 인코딩 방식
#### 예시로 '건'이라는 글자는
#### 앞의 0~18번째 벡터 중 0번째가 1, 나머지가 0
#### 중간의 0~20번째 백터 중 4번째가 1, 나머지가 0
#### 뒤의 0~27번째 벡터 중 4번째가 1, 나머지가 0
#### 위와 같이 표현이 가능하다






참고 문헌 : https://koreascience.or.kr/article/CFKO201832073079068.pdf

In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""), ".."))

import custom
import numpy

In [2]:
# 변환 작업을 함수화

def korean_encoding(text : str) :
    label = []
    chars = list(text)
    spe_ = ['<unk>',' ','.','!','?','','','','']
    for c in chars :        
        if c in spe_:
            cho  = spe_.index(c) + 19 + 1
            jung = 1
            jong = 1
            label.append(cho)
            label.append(jung)
            label.append(jong)
            continue
            
        value = ord(c) - 0xAC00
        if value < 0 or value >= 19*21*28 :
            cho  = spe_.index('<unk>') + 19 + 1
            jung = 1
            jong = 1
            label.append(cho)
            label.append(jung)
            label.append(jong)
            continue
            
        jong = value % 28 + 1
        jung = (value // 28) % 21 + 1
        cho  = (value // 28) // 21 + 1
        label.append(cho)
        label.append(jung)
        label.append(jong)
        
    label = numpy.array(label).astype(numpy.int64)
    return label

def korean_decoding(label) :
    if len(label) % 3 != 0 :
        raise Exception("data is corrupted!")
    
    text = ""
    spe_ = ['<unk>',' ','.','!','?','','','','']
    
    for i in range(len(label) // 3) :
        cho  = label[i*3]
        jung = label[i*3 + 1] 
        jong = label[i*3 + 2]

        if (cho == 0) or (jung == 0) or (jong == 0):
            break
        if cho >= 20 :
            text += spe_[cho - 20]
            continue
        
        value = ((cho - 1) * 21 + (jung - 1)) * 28 + (jong - 1) + 0xAC00
        text += chr(value)   
    return text


In [3]:


cho = ['<pad>','ㄱ','ㄲ','ㄴ','ㄷ','ㄸ','ㄹ','ㅁ','ㅂ','ㅃ','ㅅ','ㅆ','ㅇ','ㅈ','ㅉ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ','<unk>',' ','.','!','?','','','','']
jung = ['<pad>','ㅏ','ㅐ','ㅑ','ㅒ','ㅓ','ㅔ','ㅕ','ㅖ','ㅗ','ㅘ','ㅙ','ㅚ','ㅛ','ㅜ','ㅝ','ㅞ','ㅟ','ㅠ','ㅡ','ㅢ','ㅣ','','','','','','','']
jong = ['<pad>','','ㄱ','ㄲ','ㄳ','ㄴ','ㄵ','ㄶ','ㄷ','ㄹ','ㄺ','ㄻ','ㄼ','ㄽ','ㄾ','ㄿ','ㅀ','ㅁ','ㅂ','ㅄ','ㅅ','ㅆ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']

print(len(cho))
print(len(jung))
print(len(jong))

29
29
29


In [4]:
value = ord('건') - 0xAC00
chosung =  (value // 28) // 21 + 1
jungsung = (value // 28) % 21 + 1
jongsung = value % 28 + 1

print(cho[chosung])
print(jung[jungsung])
print(jong[jongsung])

ㄱ
ㅓ
ㄴ


In [5]:
value = ((cho.index('ㄱ') - 1) * 21 + (jung.index('ㅓ') - 1)) * 28 + (jong.index('ㄴ') - 1) + 0xAC00
value_char = chr(value)
print(value_char)

건


In [6]:
ori_text = '건강 하네요.A'
label = korean_encoding(ori_text)
print(label)

text = korean_decoding(label)
print(text)

[ 1  5  5  1  1 22 21  1  1 19  1  1  3  6  1 12 13  1 22  1  1 20  1  1]
건강 하네요.<unk>


In [7]:
ori_text = '건강 하네요.A'
label = custom.korean_encoding(ori_text)
print(label)

text = custom.korean_decoding(label)
print(text)

[ 1  5  5  1  1 22 21  1  1 19  1  1  3  6  1 12 13  1 22  1  1 20  1  1]
건강 하네요.<unk>
